<a href="https://colab.research.google.com/github/Mamtajangra/pytorch_tensor_autograd_practice/blob/main/pytorch_nn_module_on_diabetes2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ways to improve model accuracy
1. Building the neural network via nn module
2.using built in activation function(we use relu there)
3. use built in loss function(bceloss)
4. using built in optimizer.

In [23]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [24]:
df = pd.read_csv("/content/diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [25]:
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [26]:
df.tail(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [28]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [29]:
X = df.drop(columns="Outcome")
y = df["Outcome"]

In [30]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)


In [32]:
X_train_tensor = torch.from_numpy(X_train.values)
X_test_tensor = torch.from_numpy(X_test.values)
y_train_tensor = torch.from_numpy(y_train.values)
y_test_tensor = torch.from_numpy(y_test.values)

In [38]:
X_train_tensor = X_train_tensor.float()
y_train_tensor = y_train_tensor.float()
X_test_tensor = X_test_tensor.float()
y_test_tensor = y_test_tensor.float()


In [39]:
class MySimpleNN(nn.Module):

  def __init__(self,num_features):
    super().__init__()
    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()
  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

# important features

In [40]:
learning_rate = 0.1
epochs = 25

In [47]:
# define loss function
loss_function = nn.BCELoss()   ## ye nn module ke andar se hmne call kiya hai function binary cross entropy loss

In [48]:
# create model
model = MySimpleNN(X_train_tensor.shape[1]).float()

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  ## model.parameter()retrieves a iterator over all the trainable parameter(weights,bias)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))  ## if we dont use reshape error occur because input size 614,1 and target is 614 so use view

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 18.356298446655273
Epoch: 2, Loss: 20.201576232910156
Epoch: 3, Loss: 35.298828125
Epoch: 4, Loss: 35.298828125
Epoch: 5, Loss: 35.298828125
Epoch: 6, Loss: 35.298828125
Epoch: 7, Loss: 35.298828125
Epoch: 8, Loss: 35.298828125
Epoch: 9, Loss: 35.298828125
Epoch: 10, Loss: 35.298828125
Epoch: 11, Loss: 35.298828125
Epoch: 12, Loss: 35.298828125
Epoch: 13, Loss: 35.298828125
Epoch: 14, Loss: 35.298828125
Epoch: 15, Loss: 35.298828125
Epoch: 16, Loss: 35.298828125
Epoch: 17, Loss: 35.298828125
Epoch: 18, Loss: 35.298828125
Epoch: 19, Loss: 35.298828125
Epoch: 20, Loss: 35.298828125
Epoch: 21, Loss: 35.298828125
Epoch: 22, Loss: 35.298828125
Epoch: 23, Loss: 35.298828125
Epoch: 24, Loss: 35.298828125
Epoch: 25, Loss: 35.298828125


In [49]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.6818181872367859
